<a href="https://colab.research.google.com/github/sugiyama404/chat_bot_bert/blob/main/Similar_document_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install janome
!pip install bert-score

     |████████████████████████████████| 19.7 MB 75.7 MB/s 
     |████████████████████████████████| 59 kB 4.1 MB/s 
     |████████████████████████████████| 2.8 MB 14.4 MB/s 
     |████████████████████████████████| 3.3 MB 51.1 MB/s 
     |████████████████████████████████| 636 kB 64.8 MB/s 
     |████████████████████████████████| 52 kB 2.0 MB/s 
     |████████████████████████████████| 895 kB 64.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
import numpy as np
from janome.tokenizer import Tokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from bert_score import score

In [3]:
#わかち書き関数
def wakachi(text):

    t = Tokenizer()
    tokens = t.tokenize(text)
    docs=[]
    for token in tokens:
        docs.append(token.surface)
    return docs
 
#文書ベクトル化関数
def vecs_array(documents):
 
    docs = np.array(documents)
    vectorizer = TfidfVectorizer(analyzer=wakachi,binary=True,use_idf=False)
    vecs = vectorizer.fit_transform(docs)
    return vecs.toarray()
 

docs = ["あなたの名前は？思考戦車のタチコマ","私はタチコマです。","私は犬のことがとても好きです。","初めまして！ボクは、タチコマ！"]

#類似度行列作成
cs_array = np.round(cosine_similarity(vecs_array(docs), vecs_array(docs)),3)
print(cs_array)

[[1.    0.316 0.224 0.267]
 [0.316 1.    0.566 0.338]
 [0.224 0.566 1.    0.12 ]
 [0.267 0.338 0.12  1.   ]]


In [4]:
def similar_document_search(cand, refs, empty_flag):
    from bert_score import score
    def calc_bert_score(cands, refs):
        Precision, Recall, F1 = score(cands, refs, lang="ja", verbose=False)
        return F1.numpy().tolist()

    cands = []
    for i in refs:
        cands.append(cand)

    f1 = calc_bert_score(cands, refs)
    data = []
    for item, ref in zip(f1, refs):
        data.append([item, ref])
    data.sort(reverse=True)

    print()
    print(cand)
    for it, it2 in data:
        print(it,' ',it2)

    f_flag = True
    f_num = 0.0
    data2 = []
    for it, it2 in data:
        if f_num > it:
            break

        if f_flag:
            f_num = it
            f_num = f_num - 0.05
            f_flag = False

        data2.append([it, it2])

    if (not empty_flag) and (len(data2) >= 2):
        data2.pop(0)
    
    str = ''
    if (len(data2) >= 2):
        num = random.randrange(len(data2))
        str = data2[num][1]

    else:
        str = data2[0][1]

    return str

empty_flag = False
cand = "あなたの名前は？思考戦車のタチコマ"
refs = ["私はタチコマです。","私は犬のことがとても好きです。","初めまして！ボクは、タチコマ！"]
similar_document_search(cand, refs, empty_flag)

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/996k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).



あなたの名前は？思考戦車のタチコマ
0.7184280157089233   私はタチコマです。
0.7169045209884644   初めまして！ボクは、タチコマ！
0.6292032599449158   私は犬のことがとても好きです。


'初めまして！ボクは、タチコマ！'